In [ ]:
# Activate the following in colab : 
!pip3 install netCDF4
!apt install proj-bin libproj-dev libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/master.zip
    
import xarray as xr
import math
import numpy as np

# Single observation data assimilation experiment

#### Load BACKGROUND fields of temperature,wind, and specific humidity

In [ ]:
#url='https://thredds.met.no/thredds/dodsC/aromearcticarchive/2018/10/24/arome_arctic_full_2_5km_20181024T15Z.nc'
url='https://thredds.met.no/thredds/dodsC/metusers/maltem/GEO4902_2020/remap.varn.BG.nc'
model_background = xr.open_dataset(url)

#### Load ANALYSIS fields of temperature,wind, and specific humidity
#### This analysis has been produced by assimilating a radiosonde information at 500hPa

In [ ]:
url='https://thredds.met.no/thredds/dodsC/metusers/maltem/GEO4902_2020/remap.varn.LPB_POS2_TEMP500.nc'
model_analysis = xr.open_dataset(url)

The data is stored in the original coordinates of the model, but do not worry about that. Just assume it is some kind of vertical discretization. Remember that you can look in the metadata to find the variable names and coordinate information! 

### Below are two plotting routines which yo can use to plot horizontal and vertical sections of the model. 

In [ ]:
from matplotlib import pyplot as plt
from mpl_toolkits.basemap import Basemap

timestep=0; modellevel=59

temp_background = model_background.air_temperature[timestep,modellevel,:,:]
temp_analysis   = model_analysis.air_temperature[timestep,modellevel,:,:]

fig = plt.figure(figsize=(7.2,7.2)) #11.7
plt.subplots_adjust(left=0.05, right=0.95, top=0.90, bottom=0.05, wspace=0.15, hspace=0.05)
ax = plt.subplot(111)
m = Basemap(projection='stere', 
            boundinglat=40, lon_0=10, lat_0=78.4, 
            resolution='l', 
            llcrnrlat= 50.0, urcrnrlat=72.0, llcrnrlon=-10.0, urcrnrlon=40.0)
 
m.drawcoastlines(color='#4D5D53'); m.fillcontinents(alpha=0.2)
parallels = np.arange(0.,90,10); m.drawparallels(parallels,labels=[1,1,1,0],fontsize=10)
meridians = np.arange(0.,360.,10); m.drawmeridians(meridians,labels=[0,0,0,1],latmax=80,fontsize=10)

x, y = m(*np.meshgrid(model_background.lon,model_background.lat))

cs=m.contourf(x,y,temp_analysis,cmap=plt.cm.coolwarm,extemd='max')

cbar = m.colorbar(cs,location='bottom',pad="5%")  
cbar.set_label('Temperature in degree Celcius')                  

In [ ]:
fig = plt.figure(figsize=(7.2,7.2)) #11.7
plt.subplots_adjust(left=0.05, right=0.95, top=0.90, bottom=0.05, wspace=0.15, hspace=0.05)
ax = plt.subplot(111)


timestep=0; seclat=20; # Intersection at latitude model_analysis.lat[seclat]

tempvert_background =  np.array(model_background.air_temperature[timestep,:,seclat,:])
tempvert_analysis   =  np.array(model_analysis.air_temperature[timestep,:,seclat,:])

windvert_background=np.sqrt(np.array(model_background.x_wind[timestep,:,seclat,:]**2+model_background.y_wind[timestep,:,seclat,:]**2))
windvert_analysis  =np.sqrt(np.array(model_analysis.x_wind[timestep,:,seclat,:])**2+np.array(model_analysis.y_wind[timestep,:,seclat,:])**2)

humidvert_background =  np.array(model_background.specific_humidity[timestep,:,seclat,:])
humidvert_analysis   =  np.array(model_analysis.specific_humidity[timestep,:,seclat,:])

cs=ax.pcolor(model_background.lon, range(1,61),tempvert_background, cmap=plt.cm.coolwarm)

plt.gca().invert_yaxis()
cbar = m.colorbar(cs,location='bottom',pad="5%")  
cbar.set_label('Temperature in degree Celcius')   
#fig.colorbar(c, ax=ax1)